# Logistic  Regression

In [107]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pandas import Series, DataFrame
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings( "ignore" )

In [108]:
df = pd.read_csv("data.csv")

In [109]:
df.head()

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,X21.V1.791,135,190,229,223,192,125,55,-9,-33,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,4
1,X15.V1.924,386,382,356,331,320,315,307,272,244,...,164,150,146,152,157,156,154,143,129,1
2,X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,...,57,64,48,19,-12,-30,-35,-35,-36,5
3,X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,5
4,X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,...,4,2,-12,-32,-41,-65,-83,-89,-73,5


In [111]:
#df['y'].value_counts() 

In [112]:
df.drop(df.columns[0], axis=1, inplace=True) # Drop the 1st column from the table

In [113]:
# Created a features dataframe from the original dataframe
feature_data = df.iloc[:,0:-1]
feature_data.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X169,X170,X171,X172,X173,X174,X175,X176,X177,X178
0,135,190,229,223,192,125,55,-9,-33,-38,...,8,-17,-15,-31,-77,-103,-127,-116,-83,-51
1,386,382,356,331,320,315,307,272,244,232,...,168,164,150,146,152,157,156,154,143,129
2,-32,-39,-47,-37,-32,-36,-57,-73,-85,-94,...,29,57,64,48,19,-12,-30,-35,-35,-36
3,-105,-101,-96,-92,-89,-95,-102,-100,-87,-79,...,-80,-82,-81,-80,-77,-85,-77,-72,-69,-65
4,-9,-65,-98,-102,-78,-48,-16,0,-21,-59,...,10,4,2,-12,-32,-41,-65,-83,-89,-73


In [114]:
#Mapping Target for one V/S all logistic classification
df['y'] = df.y.map({1: 1, 2: 0, 3: 0, 4: 0, 5:0}) # CLass 1-  Recording of seizure activity, Class 0 - No seizure activity recorded
df.head(5)


,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,135,190,229,223,192,125,55,-9,-33,-38,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,0
1,386,382,356,331,320,315,307,272,244,232,...,164,150,146,152,157,156,154,143,129,1
2,-32,-39,-47,-37,-32,-36,-57,-73,-85,-94,...,57,64,48,19,-12,-30,-35,-35,-36,0
3,-105,-101,-96,-92,-89,-95,-102,-100,-87,-79,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,0
4,-9,-65,-98,-102,-78,-48,-16,0,-21,-59,...,4,2,-12,-32,-41,-65,-83,-89,-73,0


In [119]:
#Scale the data in the range of 1 to 0
min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X)
#X_train_minmax

In [120]:
def SplitData(xdata, ydata, test_s):
    """
    This function splits the features data and target data into test and training data as per the test size provided.
    """
    x_train, x_test, y_train, y_test = train_test_split(xdata,ydata,test_size = test_s ,random_state = 0)
    return x_train, x_test, y_train, y_test


In [121]:
def probability(theta, x):
    """
    This function calculates the sigmoid function.
    """
    pred = np.dot(x, theta)
    return 1 / (1 + np.exp(-pred))
 

In [142]:
def cost_function(x, y, theta):
    """
    This function calculates the logistic regression cost function.(j(0) = (1/n)[y * log(h(0) + (1-y)log(1-h(0)))] )
    """
    n = len(y)
    J = -(1 / n) * np.sum(y * np.log(probability(theta, x)) + (1 - y) * np.log(1 - probability(theta, x)))
    return J


In [154]:
#gradient function
def gradient_descent(x,y, theta,alpha, iterations):
        n = len(y)
        cost_history = [0] * iterations             # Create a array of the size equal to no.of iteration to save CostHistoty.
        for iteration in range(iterations):
                y_pred = probability(theta, x)      # Calculate h0(x)
                gradient = x.T.dot(y_pred - y)/n
                theta = theta - alpha*gradient      # update the parameter
                cost = cost_function(x,y,theta)
                cost_history[iteration] = cost
                if(cost > cost_history[iteration-1]):  # cost increases from the previous step decrease the alpha value
                     alpha = alpha*0.1
                   
        return theta, cost_history


In [152]:
def Accuracy(theta):
    """
    This function calculates the accuracy of the classification
    """
    correct = 0
    length = len(x_test)                               # Total number of samples
    prediction = (probability(theta, x_test) > 0.5)    # Get the class of the data
    _y = y_test.reshape(-1, 1)                         # Create a vector of the target data
    correct = prediction == _y                         # Compare actual and predicted class 
    my_accuracy = (np.sum(correct) / length)*100       # Acurracy = num of correctly classified / total number of samples
    print ('LR Accuracy %: ', my_accuracy)
    

In [125]:
y = df["y"]
x_train, x_test, y_train, y_test = SplitData(X_train_minmax,y , 0.30)

In [155]:
theta = np.array([0, 0, 0])                                           # Initilize theta to 0
m,n=np.shape(x_train)
y_train=np.array(y_train)
y_train=y_train.reshape(m,1)                                          # Create a vector of train target data
theta=np.zeros((n,1))
(theta, cost) = gradient_descent(x_train,y_train,theta,0.001,100)     # New parameter value set and cost history
    
MSE_train=cost[-1]
       
#test
m,n=np.shape(x_test)
y_test=np.array(y_test)
y_test=y_test.reshape(m,1)
theta=np.zeros((n,1))
(theta, cost) = gradient_descent(x_test,y_test,theta,0.001,100)
MSE_test=cost[-1]
accuracy=Accuracy(theta)
print("Training MSE  :",MSE_train)
print('Test MSE  :',MSE_test)


LR Accuracy %:  98.66666666666667
Training MSE  : 0.6855716124626019
Test MSE  : 0.6851554299236682


#### For boston dataset

In [127]:
# Load the dataset
df = pd.read_csv("boston.csv")
df.head()

,Unnamed: 0,f1,f2,f3,f4,f5,class
0,0,0.887621,0.185875,2.299474,2.299474,0.581396,healthy
1,1,0.783744,0.163794,2.299480,2.299480,0.588340,healthy
2,2,0.875873,0.154714,2.299477,2.299477,0.589802,healthy
3,3,0.881293,0.138396,2.299481,2.299481,0.592911,healthy
4,4,0.798116,0.125291,2.299481,2.299481,0.595338,healthy


In [128]:
df=df.iloc[:,1:]

In [129]:
df["Target"] = df["class"]

df['Target'] = df.Target.map({"seizure": 1, "healthy": 0, "transation": 0}) # Checking for presence of CLass 1
df.head(5)
df.drop(df.columns[5], axis=1, inplace=True)


In [130]:
df.head(5)

,f1,f2,f3,f4,f5,Target
0,0.887621,0.185875,2.299474,2.299474,0.581396,0
1,0.783744,0.163794,2.299480,2.299480,0.588340,0
2,0.875873,0.154714,2.299477,2.299477,0.589802,0
3,0.881293,0.138396,2.299481,2.299481,0.592911,0
4,0.798116,0.125291,2.299481,2.299481,0.595338,0


In [131]:
feature_data = df.iloc[:,0:5]
feature_data.head()

,f1,f2,f3,f4,f5
0,0.887621,0.185875,2.299474,2.299474,0.581396
1,0.783744,0.163794,2.299480,2.299480,0.588340
2,0.875873,0.154714,2.299477,2.299477,0.589802
3,0.881293,0.138396,2.299481,2.299481,0.592911
4,0.798116,0.125291,2.299481,2.299481,0.595338


In [132]:
#dataprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(feature_data)
#X_train_minmax

In [133]:
y = df["Target"]
x_train, x_test, y_train, y_test = SplitData(X_train_minmax,y, 0.30 )

In [134]:
theta = np.array([0, 0, 0])
m,n=np.shape(x_train)
y_train=np.array(y_train)
y_train=y_train.reshape(m,1)
theta=np.zeros((n,1))
(theta, cost) = gradient_descent(x_train,y_train,theta,0.001,100)
    
MSE_train=cost[-1]
       
#test
m,n=np.shape(x_test)
y_test=np.array(y_test)
y_test=y_test.reshape(m,1)
theta=np.zeros((n,1))
(theta, cost) = gradient_descent(x_test,y_test,theta,0.001,100)
MSE_test=cost[-1]
accuracy=Accuracy(theta)
print("Training MSE  :",MSE_train)
print('Test MSE  :',MSE_test)


LR Accuracy %:  82.22222222222221
Training MSE  : 0.6926763939481365
Test MSE  : 0.6927433437204163


##### For CHB MIT dataset

In [135]:
# Load the dataset
df = pd.read_csv("chb.csv")
df.head()

,T8-P8-1_n_peaks,T8-P8-1_n_crossings,T8-P8-1_hfd,T8-P8-1_pfd,T8-P8-1_hurst_exp,T8-P8-1_spectral_entropy,T8-P8-1_total_power,T8-P8-1_median_freq,T8-P8-1_peak_freq,T8-P8-1_hjorth_mobility,T8-P8-1_hjorth_complexity,T8-P8-1_power_1hz,T8-P8-1_power_5hz,T8-P8-1_power_10hz,T8-P8-1_power_15hz,T8-P8-1_power_20hz,seizure
0,183,153,0.094016,0.611024,0.879401,0.747444,5.090000e-10,3,1,0.002766,171.222337,0.420823,0.226051,0.128492,0.106838,0.117797,0
1,183,165,0.092275,0.611024,0.746361,0.765824,5.560000e-10,3,1,0.003211,148.493351,0.384135,0.234295,0.140616,0.115382,0.125572,0
2,186,170,0.098374,0.610385,0.687973,0.752050,5.450000e-10,3,1,0.003564,142.209197,0.389655,0.238327,0.124545,0.109260,0.138213,0
3,189,166,0.099552,0.609756,0.726399,0.754101,5.920000e-10,3,1,0.003514,140.840795,0.396989,0.232559,0.120853,0.118132,0.131467,0
4,188,167,0.101196,0.609756,0.755442,0.758187,5.830000e-10,3,1,0.003494,142.434220,0.390794,0.230665,0.125717,0.119646,0.133177,0


In [136]:
feature_data = df.iloc[:,0:-1]
feature_data.head()

,T8-P8-1_n_peaks,T8-P8-1_n_crossings,T8-P8-1_hfd,T8-P8-1_pfd,T8-P8-1_hurst_exp,T8-P8-1_spectral_entropy,T8-P8-1_total_power,T8-P8-1_median_freq,T8-P8-1_peak_freq,T8-P8-1_hjorth_mobility,T8-P8-1_hjorth_complexity,T8-P8-1_power_1hz,T8-P8-1_power_5hz,T8-P8-1_power_10hz,T8-P8-1_power_15hz,T8-P8-1_power_20hz
0,183,153,0.094016,0.611024,0.879401,0.747444,5.090000e-10,3,1,0.002766,171.222337,0.420823,0.226051,0.128492,0.106838,0.117797
1,183,165,0.092275,0.611024,0.746361,0.765824,5.560000e-10,3,1,0.003211,148.493351,0.384135,0.234295,0.140616,0.115382,0.125572
2,186,170,0.098374,0.610385,0.687973,0.752050,5.450000e-10,3,1,0.003564,142.209197,0.389655,0.238327,0.124545,0.109260,0.138213
3,189,166,0.099552,0.609756,0.726399,0.754101,5.920000e-10,3,1,0.003514,140.840795,0.396989,0.232559,0.120853,0.118132,0.131467
4,188,167,0.101196,0.609756,0.755442,0.758187,5.830000e-10,3,1,0.003494,142.434220,0.390794,0.230665,0.125717,0.119646,0.133177


In [137]:
#dataprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(feature_data)
#X_train_minmax

In [139]:
y = df["seizure"]
x_train, x_test, y_train, y_test = SplitData(X_train_minmax,y , 0.30)

In [140]:
theta = np.array([0, 0, 0])
m,n=np.shape(x_train)
y_train=np.array(y_train)
y_train=y_train.reshape(m,1)
theta=np.zeros((n,1))
(theta, cost) = gradient_descent(x_train,y_train,theta,0.001,100)
    
MSE_train=cost[-1]
       
#test
m,n=np.shape(x_test)
y_test=np.array(y_test)
y_test=y_test.reshape(m,1)
theta=np.zeros((n,1))
(theta, cost) = gradient_descent(x_test,y_test,theta,0.001,100)
MSE_test=cost[-1]
accuracy=Accuracy(theta)
print("Training MSE  :",MSE_train)
print('Test MSE  :',MSE_test)


LR Accuracy %:  98.66666666666667
Training MSE  : 0.6855716124626019
Test MSE  : 0.6851554299236682
